In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, cv2, random
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from subprocess import check_output
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.grid_search import GridSearchCV
from PIL import Image
import os
from subprocess import check_output

C:\Users\MO\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\MO\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [7]:
#读取图像
def getData():
    TRAIN_DIR = 'train/'
    TEST_DIR = 'test/'
    train_dogs =   [(TRAIN_DIR+'dog.'+str(num)+'.jpg', 1) for num in range(9375,12500)]
    train_cats =   [(TRAIN_DIR+'cat.'+str(num)+'.jpg', 0) for num in range(9375,12500)]
    #取测试集，这个地方需要提前占位，-1就是占位符。
    test_images =  [(TEST_DIR+str(num)+'.jpg', -1) for num in range(1,12501)]
    train_images = train_dogs+ train_cats
    random.shuffle(train_images)
    return train_images,test_images
train_images,test_images = getData()
#print(train_images)

In [8]:
#将图像转成dataframe的数组
def imgToDataFrame(images):
    df1 = pd.DataFrame(columns = [i for i in range(3*256)])
    df2 = pd.DataFrame(columns = [0])
    listx = []
    listy = []
    #i=1
    #print(len(images))
    for img in images:
        aimg = Image.open(img[0])
        aimg=aimg.resize((64,64),Image.ANTIALIAS)
        
        pix_val = list(aimg.getdata())
        pix_val_flat = [x for sets in pix_val for x in sets]
        #print(pix_val_flat)
        pix_val_flat = aimg.histogram()
        listx.append(pix_val_flat)
        listy.append(img[1])
        #df1=df1.append(pd.DataFrame(np.array([pix_val_flat]),columns = [i for i in range(3*64*64)]))
        #df2=df2.append(pd.DataFrame(np.array([img[1]]),columns = [0]))
        #if i>=100:break
        #i+=1
    df1 = pd.DataFrame(listx,columns = [i for i in range(256*3)])
    df2 = pd.DataFrame(listy,columns = [0])
    return df1,df2
xtrain,ytrain = imgToDataFrame(train_images)
xtest,_ = imgToDataFrame(test_images)
xtrain.head()


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0,0,0,0,0,0,0,0,0,0,...,17,14,14,24,39,65,82,26,10,22
1,2,5,5,3,3,2,5,6,6,12,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,3,4,1,2,1,3,4,...,5,5,4,5,4,4,4,7,6,7
3,0,0,0,0,0,0,0,0,0,0,...,32,21,9,11,14,14,8,19,14,22
4,5,5,15,25,40,44,52,33,51,35,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.cross_validation import cross_val_score  
from sklearn.decomposition import PCA

In [10]:
#随机森林
#forest = RandomForestClassifier(n_estimators=50)
#param_grid = { 
 #   'n_estimators': [100,200,300,400,500],  'max_features': ['auto', 'log2']
#}
#CV_forest= GridSearchCV(estimator=forest, param_grid=param_grid, cv=5,scoring='neg_log_loss')
#CV_forest.fit(xtrain,ytrain)
#Ytest = CV_forest.predict_proba(xtest)
#forest.fit(xtrain,ytrain)
#Ytest = forest.predict_proba(xtest)
#print(Ytest)
ytrain[0]

0       1
1       1
2       1
3       0
4       1
5       1
6       0
7       1
8       1
9       0
10      1
11      1
12      1
13      1
14      0
15      0
16      0
17      1
18      1
19      0
20      0
21      1
22      0
23      0
24      0
25      0
26      0
27      0
28      1
29      1
       ..
6220    0
6221    0
6222    0
6223    0
6224    1
6225    1
6226    1
6227    1
6228    1
6229    0
6230    0
6231    1
6232    1
6233    1
6234    0
6235    0
6236    0
6237    1
6238    0
6239    1
6240    1
6241    0
6242    1
6243    0
6244    0
6245    1
6246    1
6247    0
6248    1
6249    0
Name: 0, dtype: int64

In [11]:
params = {'C':[1, 10, 50, 100, 500, 1000], 'tol': [0.001, 0.0001, 0.005]}
logic = LogisticRegression()
CV_logic= GridSearchCV(estimator=logic, param_grid=params, cv=5,scoring='neg_log_loss')
print(len(xtrain))
print(ytrain.values)
CV_logic.fit(xtrain,ytrain[0])
Ytest = CV_logic.predict_proba(xtest)
#logic.fit(xtrain,ytrain)
#Ytest = logic.predict_proba(xtest)
print(Ytest)

6250
[[1]
 [1]
 [1]
 ..., 
 [0]
 [1]
 [0]]
[[ 0.63434113  0.36565887]
 [ 0.42911177  0.57088823]
 [ 0.26489276  0.73510724]
 ..., 
 [ 0.68043011  0.31956989]
 [ 0.82886768  0.17113232]
 [ 0.73307274  0.26692726]]


In [12]:
#组合成结果
result = pd.DataFrame(Ytest,columns = ['label','label1'])
result.insert(0,"id",[i for i in range(1,len(Ytest)+1)])
result = result.drop('label1',axis = 1)
result.head()
result.to_csv('dogvscatsamples.csv', index=False)